In [1]:
from typing import Any, TypeVar
from types import UnionType
import moveread.pipelines.dfy as dfy
from haskellian import either as E
from pipeteer import http, trees, ReadQueue, WriteQueue, Pipeline, SimpleQueue
from pipeteer.pipelines import GetQueue
from fastapi import FastAPI
from pydantic import TypeAdapter

T = TypeVar('T')

In [2]:
get_queue = dfy.queue_factory('queues.sqlite')
pipe = dfy.DFYPipeline()
Qout = get_queue(('output',), dfy.Output)

In [7]:
Qs = pipe.observe(get_queue)

def read_client(path, _, *, base = 'http://localhost:8000/queues'):
  url = base + '/'.join(path) + '/read'
  return http.ReadClient.validated(Any, url)

trees.flatten(trees.path_map(Qs, read_client))

Iter([(('inputval', 'Qwrapped'), <pipeteer.http.client.read.ReadClient object at 0x7f80777ab750>), (('inputval', 'wrapped', 'Qin'), <pipeteer.http.client.read.ReadClient object at 0x7f80777a9a10>), (('preprocess', 'Qwrapped'), <pipeteer.http.client.read.ReadClient object at 0x7f80777ab7d0>), (('preprocess', 'wrapped', 'preinput', 'Qin'), <pipeteer.http.client.read.ReadClient object at 0x7f80777a9ed0>), (('preprocess', 'wrapped', 'preprocess', 'extract', 'Qwrapped'), <pipeteer.http.client.read.ReadClient object at 0x7f80785c6190>), ...])